This python notebook is motivated by the text Random Walk on Wall Street. An asset that follows a random walk model is simulated. The parameters that govern the path are the starting price and percent change per second.

Run this notebook in a browser by running each cell - keyboard shortcut is Shift-Enter. Then change the parameters to the plot statement in the second cell and watch various candlestick charts.

In [54]:
import numpy as np
import pandas as pd
import datetime
from math import pi
import bokeh
from bokeh.plotting import figure, show, output_notebook
# pdave Apr 2018
# This program generates random data to create candlestick plots
# This function generates data for the given duration
# Starting Price is given by the start_price parameter
# Step size is 1 second
# Probability of Up/Down is 50%
# The result is aggregated to create candlestick chart

def df_random(**kwargs):
    volatility = kwargs['volatility']
    start_price = kwargs['start_price']
    duration_hours = kwargs['duration_hours']
    factor = 1 + volatility/100.0
    df = pd.DataFrame(pd.Series(np.random.random(3600*duration_hours)))
    df.columns = ['r']
    df['up'] = True
    D = datetime.datetime.now()
    l_D = []
    S = start_price
    l_S = []
    df.loc[df['r']<.5,'up'] = False
    n = 0
    for t in df.itertuples():
        n += 1
        if t.up:
            S = S*factor
        else:
            S = S/factor
        l_S.append(S)
        l_D.append(D + datetime.timedelta(seconds=n))
    df = pd.DataFrame({'Open': l_S, 'High': l_S, 'Low': l_S, 'Close': l_S, 'Date': l_D})
    df.index = df['Date']
    return df
# The code to plot the candlestick chart is from the following reference:
# https://bokeh.pydata.org/en/latest/docs/gallery/candlestick.html
def plot(**kwargs):
    df_r = df_random(**kwargs)
    candle_width_spec = str(kwargs['candle_width_minute']) + 'T'
    df = df_r.resample(candle_width_spec).apply({'Open':'first','High':'max','Low':'min','Close': 'last'})
    df['Date'] = df.index
    inc = df.Close > df.Open
    dec = df.Open > df.Close
    w = kwargs['candle_width_minute']*0.5*60*1000
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=900, title = "Candlestick")
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    p.segment(df.Date, df.High, df.Date, df.Low, color="black")
    p.vbar(df.Date[inc], w, df.Open[inc], df.Close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.Date[dec], w, df.Open[dec], df.Close[dec], fill_color="#F2583E", line_color="black")
    output_notebook()
    show(p)

In [55]:
plot(volatility=0.02, start_price=8000, candle_width_minute=60, duration_hours=3*24)

Loading BokehJS ...